In [4]:
import ctypes
import os
import sys
sys.path.append("..")
from cached_property import cached_property

from CANapAPI.structs import version_t
from CANapAPI.types import TAsap3Hdl
from CANapAPI.defaults import CANapAPI_dll

In [ ]:
class CANapAPI(object):
    def __init__(self, dll_path=CANapAPI_dll, *args, **kwargs):
        # Set the dll path, to lazy load later.
        self.dll_path = dll_path
    
    @cached_property
    def dll(self):
        """Lazy load the dll."""
        dll = ctypes.windll.LoadLibrary(self.dll_path)

        dll.Asap3GetVersion.argtypes = (ctypes.POINTER(version_t),)
        dll.Asap3GetVersion.restype = ctypes.c_bool
        dll.Asap3Init5.restype = ctypes.c_bool
        dll.Asap3Init5.argtypes = (
            ctypes.POINTER(TAsap3Hdl),
            ctypes.c_ulong,
            ctypes.c_char_p,
            ctypes.c_ulong,
            ctypes.c_ulong,
            ctypes.c_bool,
            ctypes.c_bool,
            ctypes.c_bool,
            ctypes.c_bool
        )
        
        # Asap3Exit
        self.dll.Asap3Exit.argtypes=(
            TAsap3Hdl,
        )
        self.dll.Asap3Exit.restype=ctypes.c_bool

        # Asap3Exit2
        self.dll.Asap3Exit2.argtypes=(
            TAsap3Hdl,
            ctypes.c_bool,
        )
        self.dll.Asap3Exit2.restype=ctypes.c_bool


        # Asap3SetInteractiveMode
        dll.Asap3SetInteractiveMode.argtypes=(
            TAsap3Hdl,
            ctypes.c_bool,
        )
        dll.Asap3SetInteractiveMode.restype=ctypes.c_bool

        # Asap3GetInteractiveMode
        dll.Asap3GetInteractiveMode.argtypes=(
            TAsap3Hdl,
            ctypes.POINTER(ctypes.c_bool),
        )
        dll.Asap3GetInteractiveMode.restype=ctypes.c_bool

        return dll
    @cached_property
    def handle(self):
        return TAsap3Hdl()
    
    @property
    def handle_ref(self):
        return self.handle.byref
    
    def Asap3SetInteractiveMode(self, interactive_mode=False):
        c_interactive_mode = ctypes.c_bool(interactive_mode)
        result = self.dll.Asap3SetInteractiveMode(
            self.handle,
            c_interactive_mode,
        )
        
    
    def Asap3GetInteractiveMode(self):
        interactive_mode = ctypes.c_bool()
        result = self.dll.Asap3GetInteractiveMode(
            self.handle,
            ctypes.byref(interactive_mode),
        )
        if result:
            return interactive_mode
        else:
            return None
    
    def Asap3GetVersion(self):
        version = version_t()
        result = dll.Asap3GetVersion(ctypes.byref(version))
        if result:
            return version
        else:
            return None
        
    def Asap3Init5(
        self,
        responseTimeout:int = 10000,
        workingDir:str = os.path.abspath("canape_tmp"),
        fifoSize:int = 8192,
        sampleSize:int = 256,
        debugMode:bool = True,
        clearDeviceList:bool = True,
        bHexmode:bool = False,
        bModalMode:bool = False,
    ):
        """Asap3Init5 Function: Modality
        
        Parameters
        ----------
        responseTimeout : int (ms)
        workingDir: str
        fifoSize: int
        sampleSize: int
        debugMode : bool
        clearDeviceList : bool
        bHexmode : bool
        bModalMode : bool
        
        Returns
        -------
        bool
            Function call success
        """
        workingDir = os.path.abspath(workingDir)
        os.makedirs(workingDir, exist_ok=True)
        
        # Convert to ctypes.
        c_responseTimeout = ctypes.c_ulong()
        c_workingDir = ctypes.c_char_p(workingDir.encode("UTF-8"))
        c_fifoSize = ctypes.c_ulong(fifoSize)
        c_sampleSize = ctypes.c_ulong(sampleSize)
        c_debugMode = ctypes.c_bool(debugMode)
        c_clearDeviceList = ctypes.c_bool(clearDeviceList)
        c_bHexmode = ctypes.c_bool(bHexmode)
        c_bModalMode = ctypes.c_bool(bModalMode)
        
        result = dll.Asap3Init5(
            self.handle.byref,
            c_responseTimeout,
            c_workingDir,
            c_fifoSize,
            c_sampleSize,
            c_debugMode,
            c_clearDeviceList,
            c_bHexmode,
            c_bModalMode,
        )
        return result   
    
    def Asap3Exit(self):
        return self.dll.Asap3Exit(self.handle)
    
    def Asap3Exit2(self, close_CANape:bool = True):
        c_close_CANape = ctypes.c_bool(close_CANape)
        return self.dll.Asap3Exit2(self.handle, c_close_CANape)
    

In [ ]:
class canapy(CANapAPI, object):
    def __init__(self, *args, **kwargs):
        if "workingDir" in kwargs:
            self.workingDir=kwargs["workingDir"]
        else:
            self.workingDir=os.path.abspath("canape_tmp")
        
        super().__init__(*args, **kwargs)
        
    @cached_property
    def version(self):
        return self.Asap3Version()
    
    def init(self, modal=True):
        return self.Asap3Init5(workingDir=self.workingDir, bModalMode=modal)
    
    def exit(self, close_canape=True):
        return self.Asap3Exit2(True)
    
    def __enter__(self):
        self.init()
        return self
    
    def __exit__(self, *args):
        print(args)

In [ ]:
type(canapy)

In [ ]:
canapy()

In [ ]:
workingDir = os.path.abspath(r"C:\Users\Public\Documents\Vector CANape 17\Examples\XCPDemo")
canape = canapy(workingDir=workingDir)

In [ ]:
canape.init()

In [ ]:
canape.Asap3Exit()

In [ ]:
with canapy(workingDir=workingDir) as canape:
    pass

In [ ]:
canape.Asap3Exit2(True)